# Testovanie hypotézy
Testovanie hypotézy pomocou Chi-Square testu. Na základe vstupného datasetu vytvoríme súhrnnú štatistiku nehôd podľa
vplyvu alkoholu a závažnosti následkov nehody. Následne pomocou vstavej funkcie knižnice *scipy* vykonáme test.
Výsledok testu vypíšeme.

## Načítanie vstupných dát
V prvom kroku načítame vstupné dáta zo súboru *accidents.pkl.gz*. Z dát extrahujeme stĺpce, ktoré sú dôležité pre test.
Zároveň odstránime nedefinované hodnoty.

In [2]:
import os
import pandas as pd
import numpy as np

# Názov súboru
filename = "accidents.pkl.gz"

# Overenie, či existuje súbor so vstupnými dátami
try:
    file_to_open = os.getcwd() + '/' + filename
    os.path.isfile(file_to_open)
except FileNotFoundError:
    exit(-1)

# Načítanie obsahu súboru
try:
    raw_data = pd.read_pickle(file_to_open, compression="gzip")
except ValueError:
    exit(-1)

# Vytvorenie DataFrame
raw_data = pd.DataFrame(raw_data)

# Extrahujeme len dáta, ktoré potrebujeme
raw_data = raw_data.loc[(raw_data['p11'] != 4) & (raw_data['p11'] != 5)][['p11', 'p13a', 'p13b']]
raw_data['p11'] = raw_data['p11'].astype('category')
raw_data['p13a'] = raw_data['p13a'].astype('category')
raw_data['p13b'] = raw_data['p13b'].astype('category')
# Odstránime neznáme hodnoty
raw_data = raw_data.replace([np.inf, -np.inf], np.nan).dropna().reset_index()
print("Počet záznamov %d" %len(raw_data.index))

Počet záznamov 485683


## Vytvorenie súhrnnej štatistiky
Vstupné dáta sú analyzované, pričom ich rozdeľujeme do 4 kategórií.
* Nehoda so silným vplyvom alkoholu a ťažkými následkami
* Nehoda so silným vplyvom alkoholu a ľahkými následkami
* Nehoda so slabým vplyvom alkoholu a ťažkými následkami
* Nehoda so slabým vplyvom alkoholu a ľahkými následkami

Takto kategorizované dáta sú vstupom samotného testu.

In [3]:
# Pole pre kategorizáciu nehôd
count_data = np.array([[0, 0],[0, 0]])

# Prechádzanie vstupných dát a triedenie nehôd
for index, row in raw_data.iterrows():
    # Vplyv alkoholu
    if row['p11'] < 7:
        # Slabý vplyv alkoholu
        # Vážnosť následkov
        if (row['p13a'] + row['p13b']) > 0:
            # Ťažké následky
            count_data[1][0] += 1
        else:
            # Ľahké následky
            count_data[1][1] += 1
    else:
        # Silný vplyv alkoholu
        # Vážnosť následkov
        if (row['p13a'] + row['p13b']) > 0:
            # Ťažké následky
            count_data[0][0] += 1
        else:
            # Ľahké následky
            count_data[0][1] += 1

# Pridanie popisov hlavičky do výsledného DataFrame
count_data=pd.DataFrame(count_data, columns=["těžké zdravotní následky", "lehké zdravotní následky"])
count_data.index=["Pod silným vlivem", "Pod slabým vlivem"]
count_data

,těžké zdravotní následky,lehké zdravotní následky
Pod silným vlivem,886,16492
Pod slabým vlivem,10777,457528


## Chi-Square test hypotézy
Posledným krokom je samotný test hypotézy pomocou vstavej funkcie *chi2_contingency* z knižnice *scipy*.
Výsledok testu je zobrazený po skončení výpočtu.

Pred spustením testu si stanovíme hypotézu, ktorú chceme dokázať a jej negáciu. Ak bude výsledná hodnota *p_value*
nižšia ako hladina významnosti 2,5% hypotéza H0 neplatí, platí pôvodná hypotéza H1.

* H0: Pokud viník nehody byl pod silným vlivem alkoholu, nedošlo častěji k těžkým zdravotním následkům.
* H1: Pokud viník nehody byl pod silným vlivem alkoholu, došlo častěji k těžkým zdravotním následkům.

In [13]:
from scipy.stats import chi2_contingency

# Výpočet samotnej p_value bez Yates korekcie výsledku
pval = chi2_contingency(count_data, correction=False)[1]
if pval < 0.025:
    print("Hypotéza H0 neplatí, platí pôvodná hypotéza H1")
else:
    print("Hypotézu H0 sme nevyvrátili, o H1 nevieme povedať nič")
print("p_value:")
pval

Hypotéza H0 neplatí, platí pôvodná hypotéza H1
p_value:


1.1538462267790811e-123